In [6]:
posts = list(reddit.subreddit('politics').hot(limit=10))

In [16]:
comments = posts[0].comments

In [23]:
for comment in comments:
    print(comment.body)

!
Remember when Barr chuckled on Sixty Minutes and said he wasn’t worried because the victors write history?
If Disingenuous was his middle name it’d be shortened to Will Dis Barr
Get fucked Mr. Barr.
Conservatives reaction on the Trump presidency appears to be like a relationship with a cheating partner who everyone knew is cheating on you. Yet when confronted by your family about your cheating spouse, you proclaim, “but she makes me feel sooo good.”
Get fucked, Barr-Trump
I’m looking forward to the day when the events at Lafayette Square for T**** photo op have daylight - Barr was also involved with that.
Will there be any repercussions for Trump's 4 years of breaking the law?
That's a funny way to spell "lying weasel"
Instead of "disingenuous" it should be "in contempt."
So many traitors in our gov; fucking unbearable.
Nothing will restore my faith in the U.S. government quite like seeing William Barr face consequences.

^...(and ^Dick ^Cheney, ^but ^I ^think ^that ^ship ^has ^saile

In [25]:
id = '9d99ku'
test_sub = reddit.submission(id=id)

In [27]:
import pprint
print(test_sub.title)
pprint.pprint(vars(test_sub))

People using vim extension, how do you do it
{'_comments': <praw.models.comment_forest.CommentForest object at 0x7f50ee17ebd0>,
 '_comments_by_id': {'t1_e5g7plt': Comment(id='e5g7plt'),
                     't1_e5ia2ue': Comment(id='e5ia2ue'),
                     't1_e5jjm5k': Comment(id='e5jjm5k')},
 '_fetched': True,
 '_reddit': <praw.reddit.Reddit object at 0x7f5108d1f9d0>,
 'all_awardings': [],
 'allow_live_comments': False,
 'approved_at_utc': None,
 'approved_by': None,
 'archived': True,
 'author': Redditor(name='WebDevBen'),
 'author_flair_background_color': None,
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_template_id': None,
 'author_flair_text': None,
 'author_flair_text_color': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_e4fjm9j',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'banned_at_utc': None,
 'banned_by': None,
 'can_gild': True,
 'can_mod_post': False,
 'category': None,
 'clicked': Fal